In [18]:
import os
import re
import sys
import numpy as np
import pandas as pd
import string
import re

import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import common_corpus, common_dictionary, datapath
import gensim


import preprocessor as p
from preprocessor.api import clean
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','rt'])

185

In [19]:
def semmatize_text(w):
    ps = PorterStemmer() 
    return ps.stem(w)
stop_words_stem = [semmatize_text(x) for x in stop_words]
stop_words.extend(stop_words_stem)
stop_words = list(dict.fromkeys(stop_words))


In [24]:
sorted(stop_words)
len(stop_words)

205

In [ ]:
DATA_DIR = "../Data"
TWEETS_PATH = os.path.join(DATA_DIR, 'tweets')
TREND_PATH = os.path.join(DATA_DIR, 'trends')
SAVE_PATH = os.path.join(DATA_DIR, 'save')
STATS_PATH = os.path.join(DATA_DIR, 'stats')
TOPICS_PATH = os.path.join(DATA_DIR, 'topics')

os.listdir(SAVE_PATH)[:5]

In [ ]:
dfs_train =  pd.read_csv(os.path.join(SAVE_PATH, "lda_train_data"), header=0, parse_dates=['trend_date'])
dfs_test =  pd.read_csv(os.path.join(SAVE_PATH, "lda_test_data"), header=0, parse_dates=['trend_date'])

In [ ]:
print(dfs_train.shape)
dfs_train.head()

In [ ]:
print(dfs_test.shape)
dfs_test.head()

# LDA

In [ ]:
def semmatize_text(text):
    ps = PorterStemmer() 
    return [ps.stem(w)  for w in text if len(w)>3]

def tokanize_text(trend_doc):
    return trend_doc.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)

def remove_stopwords(texts):
    return [word for word in texts if word not in stop_words ]

def process_lda_format(trend_doc):
    tokenized_df = tokanize_text(trend_doc)
    stemmed_dataset = tokenized_df.apply(semmatize_text)
    stemmed_dataset = stemmed_dataset.map(lambda x: remove_stopwords(x))
    return stemmed_dataset

def initialize_corpus_and_dictionary(stemmed_dataset):
    dictionary_of_words = gensim.corpora.Dictionary(stemmed_dataset)
    word_corpus = [dictionary_of_words.doc2bow(word) for word in stemmed_dataset]
    
    return word_corpus, dictionary_of_words

def lda_datasets(trend_doc):
    stemmed_dataset = process_lda_format(trend_doc)
    corpus, dictionary = initialize_corpus_and_dictionary(stemmed_dataset)
    
    return stemmed_dataset, corpus, dictionary

In [ ]:
def run_lda():
    # Model with the best coherence_value
    lda_model_16 = models.ldamodel.LdaModel(corpus = corpus, id2word=dictionary, num_topics=16, 
                                            random_state=1, update_every=1, chunksize=100,
                                            passes=50, alpha='auto', per_word_topics=True)
    
    cwd = os.getcwd()
    temp_file = datapath(os.path.join(cwd, "models/lda_model_16"))
    print('Model is saving... at', temp_file)
    lda_model_16.save(temp_file)
    
    # Compute Perplexity Score
    print('Perplexity Score: ', lda_model_16.log_perplexity(corpus)) 
    
    # Compute Coherence Score
    cohr_val = CoherenceModel(model=lda_model_16, texts=stemmed_dataset, dictionary=dictionary,
                                      coherence='c_v').get_coherence()

    print('Coherence Score: ', cohr_val)
    

In [ ]:
dfsLDA = dfs_train.loc[:,["trend","text"]]
dfsLDA.dropna(inplace=True)
trend_doc = dfsLDA.groupby(['trend'])['text'].apply(lambda x: ','.join(x)).reset_index()
trend_doc

In [ ]:
stemmed_dataset, corpus, dictionary = lda_datasets(trend_doc)

In [ ]:
stemmed_dataset

In [ ]:
run_lda()

In [ ]:
cwd = os.getcwd()
temp_file = datapath(os.path.join(cwd, "models/lda_model_16"))
temp_file

In [ ]:
# Load a potentially pretrained model from disk.
lda = models.ldamodel.LdaModel.load(temp_file)

In [ ]:
words_match = re.compile(r'\"\w+\"')

for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
#     topic_file = open(os.path.join(TOPICS_PATH, "topic-"+str(idx)+".txt"), "w+")
#     words = re.findall( words_match, topic)
#     topic_file.write( str(words) )
#     topic_file.close()

In [ ]:
target_doc = pd.read_csv(os.path.join(DATA_DIR, 'categories'), header=0)
target_doc.head(2)

In [ ]:
# TO CHECK WHAT PERCENTAGE OF THE TWEETS ARE MATCH

# df_ana = pd.read_csv(os.path.join(TWEETS_PATH,"2019-09-01_tweetsevenmorebasic.csv.bz2.bz2"), header=0)
# df_ana.shape
# print(df.shape[0]/df_ana.shape[0])